In [8]:
#importing libraries

import warnings
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec
import plotly.express as px
from pandas import plotting
import missingno as ms
from sklearn.impute import SimpleImputer
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from itertools import product
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, recall_score, precision_score, f1_score
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
warnings.filterwarnings("ignore")

In [9]:
df = pd.read_excel('Registration_to_all.xlsb')
df.head()

,Id,Company,Email,Last Name,Mobile,Website,Industry,Lead Owner Name,Lead Source,Lead Status,...,Remarks,Lead Excluded from truecaller,uppercase_Referral Code1,NClient_Code,LeadActDate,PMTD,CTD,UI,Lead Status New,Registration_Output
0,129230016355980992,NaN,NaN,LName,6.280849e+09,NaN,NaN,Tahseen Raza Ansari,Website,Follow Up,...,NaN,No,NaN,51271195.0,NaN,1,0,628084931151271195,PAN,1
1,129230016355980992,NaN,Sudarshankini7890@gmail.com,KINI,7.760890e+09,NaN,NaN,Rushikesh Patil,MobileApp,Follow Up,...,NaN,No,NaN,51271223.0,45233.401898,1,0,Sudarshankini7890@gmail.com776088956951271223,Client Activated,1
2,129230016355982000,NaN,SD726504@GMAIL.COM,DUBEY,7.905670e+09,NaN,NaN,Not contactable,MobileApp,Not Contactable 4,...,NaN,No,NaN,51271178.0,NaN,1,0,SD726504@GMAIL.COM790567024851271178,Personal/Documents/Esign,1
3,129230016355982000,NaN,VISHAL.D.1628.VS@GMAIL.COM,LName,9.566273e+09,NaN,NaN,Not contactable,UAC,Not Contactable 4,...,NaN,No,NaN,51271197.0,NaN,1,0,VISHAL.D.1628.VS@GMAIL.COM956627348951271197,PAN,1
4,129230016355982000,NaN,anandsinghkumar8292@GMAIL.COM,LName,8.292662e+09,NaN,NaN,Iram Shaikh,Website,Follow Up,...,NaN,No,51690971,51271280.0,NaN,1,0,anandsinghkumar8292@GMAIL.COM829266199151271280,Bank,1


In [10]:
df.shape

(589984, 188)

In [11]:

def calc_iv_num(df2,feature, target):
    df=df2.copy()
    print(feature)
    lst = []
    df[feature] = df[feature].replace([np.inf,-np.inf],np.nan)
    df[feature] = df[feature].fillna(0)
    df[feature] = pd.to_numeric(df[feature],errors='coerce')
    #Split conitnous variables into max 10 bins based on percentile
    df[feature+'_bin'] = pd.qcut(df[feature],q=10,duplicates='drop')
    for i in range(df[feature+'_bin'].nunique()):
        val = list(df[feature+'_bin'].unique())[i]
        lst.append([feature,
                   val,
                   df[df[feature+'_bin']==val].count()[feature],
                   df[(df[feature+'_bin']==val) & (df[target]=='Good')].count()[feature],
                   df[(df[feature+'_bin']==val) & (df[target]=='Bad')].count()[feature]])
    data = pd.DataFrame(lst,columns=['Variable','Value','All','Goods','Bads'])
    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bads'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bads'])/(data['All'].sum() - data['Bads'].sum())
    data['Distribution Bad'] = data['Bads']/data['Bads'].sum()
    data['WoE']=np.log(data['Distribution Good']/data['Distribution Bad'])
    data = data.replace({'WoE':{np.inf:0,-np.inf:0}})
    data['IV_bin']=data['WoE']*(data['Distribution Good'] - data['Distribution Bad'])
    data = data.sort_values(by = ['Variable','Value'],ascending = [True,True])
    data.index = range(len(data.index))
    data['IV']=data['IV_bin'].sum()
    
    return data

def calc_iv_cat(df2,feature, target):
    df=df2.copy()
    print(feature)
    lst = []
    # Ignore features having more than 100 unique values
    if df[feature].nunique()>100:
        return pd.DataFrame()
    df[feature] = df[feature].fillna(0)
    df[feature+'_bin'] = df[feature]
    for i in range(df[feature+'_bin'].nunique()):
        val = list(df[feature+'_bin'].unique())[i]
        lst.append([feature,
                   val,
                   df[df[feature+'_bin']==val].count()[feature],
                   df[(df[feature+'_bin']==val) & (df[target]=='Good')].count()[feature],
                   df[(df[feature+'_bin']==val) & (df[target]=='Bad')].count()[feature]])
    data = pd.DataFrame(lst,columns=['Variable','Value','All','Goods','Bads'])
    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bads'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bads'])/(data['All'].sum() - data['Bads'].sum())
    data['Distribution Bad'] = data['Bads']/data['Bads'].sum()
    data['WoE']=np.log(data['Distribution Good']/data['Distribution Bad'])
    data = data.replace({'WoE':{np.inf:0,-np.inf:0}})
    data['IV_bin']=data['WoE']*(data['Distribution Good'] - data['Distribution Bad'])
    #data = data.sort_values(by = ['Variable','Value'],ascending = [True,True])
    data.index = range(len(data.index))
    data['IV']=data['IV_bin'].sum()
    
    return data


def iterate_vars_iv(df,target):
    data = pd.DataFrame()
#     df=df[,2:]
    varlist = list(df.columns)
   # varlist = varlist[2:]
    for i in varlist:
        if df[str(i)].dtype == 'int' or  df[str(i)].dtype == 'float':
            data = pd.concat([data,calc_iv_num(df,str(i),target)])
        else:
            data = pd.concat([data,calc_iv_cat(df,str(i),target)])
    return data
              


In [12]:
import numpy as np
df['target_final'] = np.where(df.Registration_Output == 0, "Bad","Good")

In [13]:
iv_output = iterate_vars_iv(df,'target_final')
iv_output.to_csv('IV_Output_registration_novdec.csv')

Id
Company
Email
Last Name
Mobile
Website
Industry
Lead Owner Name
Lead Source
Lead Status
Lead Owner
Created Time
Is Converted?
Tasks Involved
Events Involved
Calls Involved
Activities Involved
First Name
Created By
Modified By
Modified Time
Last Activity Time
Lead Product
Lead Sub-Product
Lead Sub Status
Campaign
Rejected Reason
Disposition
Converted Account
Converted Contact
Phone
Search Partner Network
Referral Code
Client Code
Result URL
GCLID
utm_campaign
utm_content
utm_source
utm_term
utm_medium
vt_adgroupid
vt_placement
vt_campaignid
vt_network
vt_creative
vt_device
vt_keyword
vt_adposition
Permanent Address Proof
Correspondence Address Proof
PAN Card
Photograph
Income Proof
Bank Account Proof
eSign Eligibility
KYC Option
Payment Mode
Photograph Approval
Income Proof Approval
Bank Account Proof Approval
Address Approval
PAN Card Approval
eSign Done
Address Proof Rejection
Bank Account Proof Rejection
Photograph Rejection
PAN Card Rejection
Income Proof Rejection
Signature Reje

In [14]:
# Function to check missing values, unique values, and calculate percentages
def check_missing_unique(data):
    summary = pd.DataFrame({
        'Column': data.columns,
        'Missing Values': data.isnull().sum(),
        'Unique Values': data.nunique(),
        'Missing Value Percentage': data.isnull().sum() / len(data) * 100,
        'Unique Value Percentage': data.nunique() / len(data) * 100
    })
    return summary

# Call the function on your dataset
summary = check_missing_unique(df)

# Print the summary
print(summary)

summary.to_csv('summary_registration.csv')

                                  Column  Missing Values  Unique Values  \
Id                                    Id               0         209397   
Company                          Company          589984              0   
Email                              Email           93220         496431   
Last Name                      Last Name               1          83357   
Mobile                            Mobile           54377         535413   
...                                  ...             ...            ...   
CTD                                  CTD               0              2   
UI                                    UI               2         589955   
Lead Status New          Lead Status New               0              6   
Registration_Output  Registration_Output               0              2   
target_final                target_final               0              2   

                     Missing Value Percentage  Unique Value Percentage  
Id                        